In [20]:
import yfinance as yf
import pandas as pd
# pip install yfinance

In [6]:
spy=yf.Ticker('SPY')
type(spy)

yfinance.ticker.Ticker

In [7]:
spy.info

{'exchange': 'PCX',
 'shortName': 'SPDR S&P 500',
 'longName': 'SPDR S&P 500 ETF Trust',
 'exchangeTimezoneName': 'America/New_York',
 'exchangeTimezoneShortName': 'EST',
 'isEsgPopulated': False,
 'gmtOffSetMilliseconds': '-18000000',
 'quoteType': 'ETF',
 'symbol': 'SPY',
 'messageBoardId': 'finmb_6160262',
 'market': 'us_market',
 'annualHoldingsTurnover': None,
 'enterpriseToRevenue': None,
 'beta3Year': 1,
 'profitMargins': None,
 'enterpriseToEbitda': None,
 '52WeekChange': None,
 'morningStarRiskRating': None,
 'forwardEps': None,
 'revenueQuarterlyGrowth': None,
 'fundInceptionDate': 727660800,
 'annualReportExpenseRatio': None,
 'totalAssets': 367353823232,
 'bookValue': None,
 'fundFamily': 'SPDR State Street Global Advisors',
 'lastFiscalYearEnd': None,
 'netIncomeToCommon': None,
 'trailingEps': None,
 'lastDividendValue': None,
 'SandP52WeekChange': None,
 'priceToBook': None,
 'nextFiscalYearEnd': None,
 'yield': 0.016,
 'mostRecentQuarter': None,
 'enterpriseValue': None

In [9]:
spy.institutional_holders

,0,1
0,Net Assets,367.35B
1,NAV,395.94
2,PE Ratio (TTM),22.01
3,Yield,1.60%
4,YTD Daily Total Return,-15.77%
5,Beta (5Y Monthly),1.00
6,Expense Ratio (net),0.09%
7,Inception Date,1993-01-22


In [13]:
spy.earnings_history # need to fetch from bloomberg?

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise(%)
0,SPY,SPDR S&P 500 ETF Trust,"Nov 29, 2017, 12 PMEST",NaN,NaN,NaN
1,SPY,SPDR S&P 500 ETF Trust,"Aug 15, 2017, 1 PMEDT",NaN,NaN,NaN
2,SPY,SPDR S&P 500 ETF Trust,"May 15, 2017, 1 PMEDT",NaN,NaN,NaN
3,SPY,SPDR S&P 500 ETF Trust,"Feb 15, 2017, 12 PMEST",NaN,NaN,NaN
4,SPY,SPDR S&P 500 ETF Trust,"Nov 28, 2016, 12 PMEST",NaN,NaN,NaN
5,SPY,SPDR S&P 500 ETF Trust,"Aug 15, 2016, 1 PMEDT",NaN,NaN,NaN
6,SPY,SPDR S&P 500 ETF Trust,"May 15, 2016, 1 PMEDT",NaN,NaN,NaN
7,SPY,SPDR S&P 500 ETF Trust,"Feb 15, 2016, 12 PMEST",NaN,NaN,NaN
8,SPY,SPDR S&P 500 ETF Trust,"May 15, 2006, 12 AMEDT",2.11,NaN,NaN
9,SPY,SPDR S&P 500 ETF Trust,"Feb 15, 2006, 12 AMEST",2.02,NaN,NaN


In [27]:
spy.option_chain('2022-10-29')

ValueError: Expiration `2022-10-29` cannot be found. Available expiration are: [2022-12-01, 2022-12-02, 2022-12-05, 2022-12-06, 2022-12-07, 2022-12-09, 2022-12-12, 2022-12-14, 2022-12-16, 2022-12-23, 2022-12-30, 2023-01-06, 2023-01-20, 2023-02-17, 2023-03-17, 2023-03-31, 2023-04-21, 2023-06-16, 2023-06-30, 2023-09-15, 2023-09-29, 2023-12-15, 2024-01-19, 2024-06-21, 2024-12-20, 2025-01-17]

In [30]:
FetchedSpy=spy.history(period='max')

In [29]:
print(spy.history(period='max'))

                                 Open        High         Low       Close  \
Date                                                                        
1993-01-29 00:00:00-05:00   25.352051   25.352051   25.225921   25.334032   
1993-02-01 00:00:00-05:00   25.352037   25.514204   25.352037   25.514204   
1993-02-02 00:00:00-05:00   25.496194   25.586286   25.442138   25.568268   
1993-02-03 00:00:00-05:00   25.604304   25.856563   25.586286   25.838545   
1993-02-04 00:00:00-05:00   25.928644   26.000718   25.640348   25.946663   
...                               ...         ...         ...         ...   
2022-11-23 00:00:00-05:00  399.549988  402.929993  399.309998  402.420013   
2022-11-25 00:00:00-05:00  401.829987  402.910004  401.540009  402.329987   
2022-11-28 00:00:00-05:00  399.089996  400.809998  395.109985  395.910004   
2022-11-29 00:00:00-05:00  396.049988  397.299988  393.299988  395.230011   
2022-11-30 00:00:00-05:00  395.489990  407.679993  393.480011  407.679993   

In [32]:
FetchedSpy.to_csv("SPYoutPut.csv")